In [38]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils import get_column_letter

In [39]:
# Синтетичні дані
np.random.seed(42)
df = pd.DataFrame({
    "Sales": np.random.randint(50, 200, 10),
    "Discrep, $": np.random.uniform(-15, 15, 10),
    "Discrep, %": np.random.uniform(-15, 15, 10)
})

def highlight_index(s):
    return ["background-color: lightblue; color: black; font-weight: bold; border: 1px solid; text-align: center" for v in s]

def highlight_header(s):
    return ["background-color: lightblue; color: black; font-weight: bold; text-align: center" for v in s]

def style_main_page(df):
    df = df.round(2)
    df_style = df.style\
        .apply_index(highlight_index, axis=0)\
        .apply_index(highlight_header, axis=1)\
        .background_gradient("RdYlGn", vmax=10, vmin=-10, subset=["Discrep, $"])\
        .background_gradient("RdYlGn", vmax=10, vmin=-10, subset=["Discrep, %"])\
        .map(lambda x: 'color: white; background-color: white' if pd.isnull(x) else 'text-align: center')
    return df_style

df_1 = style_main_page(df)
df_1

,Sales,"Discrep, $","Discrep, %"
0,152,-4.990000,-14.790000
1,142,-10.710000,-14.310000
2,64,4.530000,0.740000
3,156,-13.310000,-3.000000
4,121,6.660000,-13.600000
5,70,13.160000,14.210000
6,152,-14.980000,-8.020000
7,171,14.770000,-12.280000
8,124,3.520000,3.550000
9,137,3.350000,-3.530000


In [40]:
df_2 = df.style.bar(subset=["Discrep, $", "Discrep, %"], align="zero", color=["#d65f5f", "#5fba7d"])
df_2

,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,-14.788011
1,142,-10.713995,-14.308127
2,64,4.526654,0.743240
3,156,-13.307653,-3.004171
4,121,6.659963,-13.600030
5,70,13.156581,14.212666
6,152,-14.976637,-8.016860
7,171,14.766347,-12.281807
8,124,3.524445,3.551580
9,137,3.349595,-3.526140


In [41]:
df_3 = df.style.background_gradient(cmap="coolwarm", subset=["Discrep, %"])
df_3

,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,-14.788011
1,142,-10.713995,-14.308127
2,64,4.526654,0.743240
3,156,-13.307653,-3.004171
4,121,6.659963,-13.600030
5,70,13.156581,14.212666
6,152,-14.976637,-8.016860
7,171,14.766347,-12.281807
8,124,3.524445,3.551580
9,137,3.349595,-3.526140


In [42]:
df_4 = df.style.highlight_min(color="lightcoral").highlight_max(color="lightgreen")
df_4

,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,-14.788011
1,142,-10.713995,-14.308127
2,64,4.526654,0.743240
3,156,-13.307653,-3.004171
4,121,6.659963,-13.600030
5,70,13.156581,14.212666
6,152,-14.976637,-8.016860
7,171,14.766347,-12.281807
8,124,3.524445,3.551580
9,137,3.349595,-3.526140


In [43]:
def up_down_color(val):
    color = "green" if val > 0 else "red"
    return f"color: {color}; font-weight: bold; text-align: center"

df_5 = df.style.applymap(up_down_color, subset=["Discrep, $", "Discrep, %"])
df_5

/var/folders/7x/q4zhf1wx08z6l2f4bqz5b8dc0000gn/T/ipykernel_5208/1754113219.py:5: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_5 = df.style.applymap(up_down_color, subset=["Discrep, $", "Discrep, %"])


,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,-14.788011
1,142,-10.713995,-14.308127
2,64,4.526654,0.743240
3,156,-13.307653,-3.004171
4,121,6.659963,-13.600030
5,70,13.156581,14.212666
6,152,-14.976637,-8.016860
7,171,14.766347,-12.281807
8,124,3.524445,3.551580
9,137,3.349595,-3.526140


In [44]:
df_6 = df.style.set_properties(**{
    "font-size": "12pt",
    "border": "1px solid black",
    "text-align": "center"
})
df_6

,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,-14.788011
1,142,-10.713995,-14.308127
2,64,4.526654,0.743240
3,156,-13.307653,-3.004171
4,121,6.659963,-13.600030
5,70,13.156581,14.212666
6,152,-14.976637,-8.016860
7,171,14.766347,-12.281807
8,124,3.524445,3.551580
9,137,3.349595,-3.526140


In [45]:
df_7 =df.style.text_gradient(subset=["Discrep, %"], cmap="rainbow")
df_7

,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,-14.788011
1,142,-10.713995,-14.308127
2,64,4.526654,0.743240
3,156,-13.307653,-3.004171
4,121,6.659963,-13.600030
5,70,13.156581,14.212666
6,152,-14.976637,-8.016860
7,171,14.766347,-12.281807
8,124,3.524445,3.551580
9,137,3.349595,-3.526140


In [46]:
def pass_fail(val):
    return "✔️" if val > 0 else "❌"

df_8 = df.style.format({"Discrep, %": pass_fail})
df_8

,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,❌
1,142,-10.713995,❌
2,64,4.526654,✔️
3,156,-13.307653,❌
4,121,6.659963,❌
5,70,13.156581,✔️
6,152,-14.976637,❌
7,171,14.766347,❌
8,124,3.524445,✔️
9,137,3.349595,❌


In [47]:
df_9 = df.style.highlight_between(subset=["Discrep, %"], left=-5, right=5, color="green")
df_9

,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,-14.788011
1,142,-10.713995,-14.308127
2,64,4.526654,0.743240
3,156,-13.307653,-3.004171
4,121,6.659963,-13.600030
5,70,13.156581,14.212666
6,152,-14.976637,-8.016860
7,171,14.766347,-12.281807
8,124,3.524445,3.551580
9,137,3.349595,-3.526140


In [48]:
df_10 = df.style.applymap(lambda v: "background-color: green" if v < 0 else "")
df_10

/var/folders/7x/q4zhf1wx08z6l2f4bqz5b8dc0000gn/T/ipykernel_5208/854136137.py:1: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_10 = df.style.applymap(lambda v: "background-color: green" if v < 0 else "")


,Sales,"Discrep, $","Discrep, %"
0,152,-4.988742,-14.788011
1,142,-10.713995,-14.308127
2,64,4.526654,0.743240
3,156,-13.307653,-3.004171
4,121,6.659963,-13.600030
5,70,13.156581,14.212666
6,152,-14.976637,-8.016860
7,171,14.766347,-12.281807
8,124,3.524445,3.551580
9,137,3.349595,-3.526140


In [49]:
# Відкриваємо ExcelWriter (двигун openpyxl), щоб писати кілька DataFrame в один файл
with pd.ExcelWriter('data/excel_result.xlsx', engine='openpyxl') as writer:
    # Записуємо df_1 у лист "Stat Check", залишаємо індекс
    df_1.to_excel(writer, sheet_name='Stat Check', index=True)
    df_2.to_excel(writer, sheet_name='Country Check', index=True)
    df_3.to_excel(writer, sheet_name='Price Point Check', index=False)
    df_4.to_excel(writer, sheet_name='P-value Check', index=False)
    df_5.to_excel(writer, sheet_name='Discrepancy Check', index=False)
    df_6.to_excel(writer, sheet_name='Styled Check', index=False)
    df_7.to_excel(writer, sheet_name='Gradient Check', index=False)
    df_8.to_excel(writer, sheet_name='Pass Fail Check', index=False)
    df_9.to_excel(writer, sheet_name='Highlight Range Check', index=False)
    df_10.to_excel(writer, sheet_name='Conditional Format Check', index=False)
    
    # Проходимося по всіх створених листах у файлі
    for sheet_name in writer.sheets:
        worksheet = writer.sheets[sheet_name]
        # Для кожної колонки визначаємо потрібну ширину
        for col in worksheet.columns:
            max_length = 0
            # Беремо літеру колонки (A, B, C, ...)
            col_letter = get_column_letter(col[0].column)
            # Перевіряємо всі клітинки у цій колонці
            for cell in col:
                try:
                    if cell.value:
                        # Рахуємо максимальну довжину тексту в клітинках
                        max_length = max(max_length, len(str(cell.value)))
                except:
                    pass
            # Задаємо ширину колонки: довжина тексту + 2 для "повітря"
            worksheet.column_dimensions[col_letter].width = max_length + 2